In [1]:
import pandas as pd
import numpy as np
import torch
import torch
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
from torch.nn import functional as F
import pickle

In [2]:
# For ZY no csv read
#with open('mtgdata.pickle', 'rb') as file:
#    mtg_df=pickle.load(file)
mtg_df=pd.read_csv('mtg_data.csv', index_col=0)
mtg_df=mtg_df.dropna(subset=['text_prompt', 'card_description'])
mtg_df

,name,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,rarity,flavor_text,text,text_prompt,card_description
0,Fury Sliver,{5}{R},6.0,Creature — Sliver,All Sliver creatures have double strike.,3,3,['R'],['R'],[],uncommon,"""A rift opened, and our arrows were abruptly s...",Fury Sliver: [SEP] {5}{R} [SEP] Creature — Sli...,Fury Sliver: [SEP] {5}{R},Creature — Sliver [SEP] All Sliver creatures h...
1,Kor Outfitter,{W}{W},2.0,Creature — Kor Soldier,"When ~ enters the battlefield, you may attach ...",2,2,['W'],['W'],[],common,"""We take only what we need to survive. Believe...",Kor Outfitter: [SEP] {W}{W} [SEP] Creature — K...,Kor Outfitter: [SEP] {W}{W},Creature — Kor Soldier [SEP] When ~ enters the...
2,Spirit,NaN,0.0,Token Creature — Spirit,Flying,1,1,['W'],['W'],[Flying],common,NaN,Spirit: [SEP] [SEP] Token Creature — Spirit [...,Spirit: [SEP],Token Creature — Spirit [SEP] Flying
3,Siren Lookout,{2}{U},3.0,Creature — Siren Pirate,"Flying\nWhen ~ enters the battlefield, it expl...",1,2,['U'],['U'],"[Flying, Explore]",common,NaN,Siren Lookout: [SEP] {2}{U} [SEP] Creature — S...,Siren Lookout: [SEP] {2}{U},Creature — Siren Pirate [SEP] Flying\nWhen ~ e...
4,Web,{G},1.0,Enchantment — Aura,Enchant creature (Target a creature as you cas...,NaN,NaN,['G'],['G'],[Enchant],rare,NaN,Web: [SEP] {G} [SEP] Enchantment — Aura [SEP] ...,Web: [SEP] {G},Enchantment — Aura [SEP] Enchant creature (Tar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85059,Celestine Reef,NaN,0.0,Plane — Luvion,Creatures without flying or islandwalk can't a...,NaN,NaN,[],[],[],rare,NaN,Celestine Reef: [SEP] [SEP] Plane — Luvion [S...,Celestine Reef: [SEP],Plane — Luvion [SEP] Creatures without flying ...
85060,Horned Troll,{2}{G},3.0,Creature — Troll,{G}: Regenerate ~.,2,2,['G'],['G'],[],common,Sword hilts jut from some trolls' bodies where...,Horned Troll: [SEP] {2}{G} [SEP] Creature — Tr...,Horned Troll: [SEP] {2}{G},Creature — Troll [SEP] {G}: Regenerate ~.
85061,Faerie Bladecrafter,{2}{B},3.0,Creature — Faerie Rogue,Flying\nWhenever one or more Faeries you contr...,2,2,['B'],['B'],[Flying],rare,NaN,Faerie Bladecrafter: [SEP] {2}{B} [SEP] Creatu...,Faerie Bladecrafter: [SEP] {2}{B},Creature — Faerie Rogue [SEP] Flying\nWhenever...
85062,Exultant Skymarcher,{1}{W}{W},3.0,Creature — Vampire Soldier,Flying,2,3,['W'],['W'],[Flying],common,"""We have come at last to this holiest of holy ...",Exultant Skymarcher: [SEP] {1}{W}{W} [SEP] Cre...,Exultant Skymarcher: [SEP] {1}{W}{W},Creature — Vampire Soldier [SEP] Flying


In [36]:
types=['Creature', 'Sorcery', 'Artifact', 'Enchantment', 'Instant', 'Land']
type_list={'Name':[],
           'Type':[]}
for idx, row in mtg_df.iterrows():
    for word in row['type_line'].split():
        if word in types:
            type_list['Name'].append(row['name'])
            type_list['Type'].append(word)

In [42]:
type_data=pd.DataFrame(type_list).drop_duplicates()
for type in types:
    type_data[type]=type_data['Type']==type
type_data

,Name,Type,Creature,Sorcery,Artifact,Enchantment,Instant,Land
0,Fury Sliver,Creature,True,False,False,False,False,False
1,Kor Outfitter,Creature,True,False,False,False,False,False
2,Spirit,Creature,True,False,False,False,False,False
3,Siren Lookout,Creature,True,False,False,False,False,False
4,Web,Enchantment,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...
83968,Born to Drive,Enchantment,False,False,False,True,False,False
83978,Stern Mentor,Creature,True,False,False,False,False,False
83981,Youthful Valkyrie,Creature,True,False,False,False,False,False
83982,Fallaji Vanguard,Creature,True,False,False,False,False,False


In [44]:
type_data.drop(['Type'], axis=1)

,Name,Creature,Sorcery,Artifact,Enchantment,Instant,Land
0,Fury Sliver,True,False,False,False,False,False
1,Kor Outfitter,True,False,False,False,False,False
2,Spirit,True,False,False,False,False,False
3,Siren Lookout,True,False,False,False,False,False
4,Web,False,False,False,True,False,False
...,...,...,...,...,...,...,...
83968,Born to Drive,False,False,False,True,False,False
83978,Stern Mentor,True,False,False,False,False,False
83981,Youthful Valkyrie,True,False,False,False,False,False
83982,Fallaji Vanguard,True,False,False,False,False,False


In [51]:
card_type=torch.tensor(type_data.drop(['Name', 'Type'], axis=1).values, dtype=torch.long)
card_type

tensor([[1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        ...,
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0]])

In [10]:
ls = [text[0] for text in mtg_df['type_line'].str.split(' ')]
freq={}
for item in ls:
   if item in freq:
      freq[item] += 1
   else:
      freq[item] = 1
print(freq)

{'Creature': 28218, 'Token': 1874, 'Enchantment': 7642, 'Sorcery': 8697, 'Basic': 3642, 'Artifact': 8157, 'Legendary': 9344, 'Instant': 8839, 'Land': 4580, 'Card': 307, 'Snow': 181, 'Conspiracy': 26, 'Plane': 211, 'Emblem': 109, 'Stickers': 49, 'Phenomenon': 22, 'Vanguard': 119, 'World': 57, 'Tribal': 130, 'Scheme': 58, 'Hero': 21, 'Summon': 16, 'Host': 29, 'Ongoing': 12, 'Elemental': 2, 'Eaturecray': 1, 'instant': 1, 'Dungeon': 3, 'Tolkien': 1, 'Planeswalker': 2, 'Scariest': 1}
